In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio
import os
import sys
import time
import math
import xarray as xr
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from Water_Blance_Model import mYWBMnlS_RE, abcdnlS_RE, DWBMnlS_RE, GmYWBM_RE
from Rewrite_Func import nash_sutcliffe_efficiency, relative_error, kling_gupta_efficiency
from numba import float64, njit
from numba.experimental import jitclass
from netCDF4 import Dataset
from scipy.spatial.distance import mahalanobis
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from scipy.ndimage import median_filter
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import trange
from scipy.stats import boxcox

## 逐流域算 AIC/BIC & 性能指标 → 判定显著性

下面给出 **Step 2：逐流域算 AIC/BIC ＋ 性能指标并判断显著性** 的一套 *可直接落地* 的流程和示例代码框架。你只需把自己的观测和模拟结果填进去即可运行。整套流程分成 ❶–❻ 六步；其中 ❹–❻ 是可选进阶，用来给出统计显著性和“优胜概率”判定。

---

## ❶ 准备数据

| 变量            | 说明                  | 建议格式                              |
| ------------- | ------------------- | --------------------------------- |
| `Q_obs`       | 观测日（或月）径流时间序列       | `numpy.ndarray` 或 `pandas.Series` |
| `Q_mod_A/B/C` | 三个模型对同一时段的模拟        | 同上                                |
| `n`           | 有效样本长度（去掉缺测后的点数）    | `int`                             |
| `k_A/B/C`     | 各模型自由参数个数（含误差方差 σ²） | `int`                             |

> 💡 **注意**：如果你对残差做 Box–Cox 变换或 log 变换，记得把 **λ**（Box–Cox 参数）也算进 `k`。

---

## ❷ 计算对数似然 `lnL`

常用假设：

* 经过变换的残差 \~ N (μ, σ²)，独立同分布。
* 给定 σ，以残差平方和 (SSR) 直接求对数似然：

$$
\ln L = -\frac{n}{2}\left[\ln(2\pi\sigma^2)\right]-\frac{1}{2\sigma^2}\sum_{t=1}^{n} (e_t-μ)^2
$$

其中

$$
\sigma^2 = \frac{1}{n}\sum_{t=1}^{n} (e_t)^2
$$

### Python 函数示例

```python
import numpy as np

def loglik_gaussian(residuals: np.ndarray) -> float:
    n = residuals.size
    sigma2 = residuals.var(ddof=0)
    return -0.5 * n * (np.log(2*np.pi*sigma2) + 1)

def loglik_gaussian_with_mu(residuals):
    n = residuals.size
    mu_hat = residuals.mean()
    sigma2_hat = ((residuals - mu_hat) ** 2).mean()
    lnL = -0.5 * n * (np.log(2*np.pi*sigma2_hat) + 1)
    return lnL, mu_hat, sigma2_hat
```

当你考虑 μ 和 σ² 都为模型参数时，务必更新参数个数k:

$$
k = k_{模型参数}+2
$$

其中“2”来自μ和σ²

---

## ❸ 计算 AIC / BIC 及 ΔAIC / ΔBIC

```python
def information_criteria(lnL: float, k: int, n: int):
    AIC = 2*k - 2*lnL
    BIC = k*np.log(n) - 2*lnL
    return AIC, BIC
```

* **每个流域**算三组 (A, B, C)。
* 找到该流域 AIC/BIC 最小的模型，记为 *best*；其余模型的

  $$
  \Delta\text{AIC}_m = \text{AIC}_m - \text{AIC}_{best}
  $$

  $$
  \Delta\text{BIC}_m = \text{BIC}_m - \text{BIC}_{best}
  $$

判定规则（常用阈值）：

| ΔAIC/ΔBIC | 0–2    | 2–4 | 4–10 | > 10 |
| --------- | ------ | --- | ---- | ---- |
| 解释        | 证据几乎相同 | 略好  | 明显好  | 极其显著 |

---

## ❹ 计算性能指标（NSE、KGE…）

```python
def nse(obs, sim):
    return 1 - (((obs - sim) ** 2).sum() /
                ((obs - obs.mean()) ** 2).sum())

def kge(obs, sim):
    r   = np.corrcoef(obs, sim)[0,1]
    alpha = sim.std() / obs.std()
    beta  = sim.mean() / obs.mean()
    return 1 - np.sqrt((r-1)**2 + (alpha-1)**2 + (beta-1)**2)
```

保存到同一个 `DataFrame`，后面可直接做差：

$$
\Delta\text{NSE}_{C-B} = \text{NSE}_C - \text{NSE}_B
$$

---

## ❺ 判定差异显著性（两条思路，至少选其一）

### A. 信息准则阈值法（快速）

* 在每个流域：若 **ΔBIC<4**，视为“差别不足”；
  若 **ΔBIC ≥ 4** 且 C 最佳 → 记作 “C 显著优”；
  若 C 不是最佳 → “C 不优”。
* 优缺点：简单，但只依赖似然，不看 NSE/KGE。

### B. Bootstrapped 性能差异置信区间（更稳健）

1. **对齐日期**：保持 `obs` 与 `sim` 同长。
2. **块自助法 (block bootstrap)**：

   ```python
   import random
   block = 30  # 30 天块
   n_bs  = 1000
   diff  = []
   for _ in range(n_bs):
       idx = []
       while len(idx) < len(obs):
           start = random.randint(0, len(obs)-block)
           idx.extend(range(start, start+block))
       idx = idx[:len(obs)]
       diff.append(nse(obs[idx], simC[idx]) -
                   nse(obs[idx], simB[idx]))
   lo, hi = np.percentile(diff, [2.5, 97.5])
   signif = (lo > 0) or (hi < 0)   # True→显著
   ```
3. 统计 2003 个流域里 `signif==True` 且 C 优的比例。

---

## ❻ （可选）转化为 **后验模型概率 PMP** 以便后续 BMA

模型权重：

$$
w_m = \exp\!\left(-\frac{1}{2}\,\Delta\text{BIC}_m\right),\qquad
\text{PMP}_m = \frac{w_m}{\sum_{j} w_j}
$$

一步 Python：

```python
weights = np.exp(-0.5 * delta_BIC_array)
PMP     = weights / weights.sum(axis=1, keepdims=True)
```

到此，一个包含字段

\| basin\_id | best\_model | ΔAIC\_A/B/C | ΔBIC\_A/B/C | NSE\_A/B/C | KGE\_A/B/C | signif\_CvsB | PMP\_A/B/C |

的总表（`DataFrame` 或 `Parquet`）就准备好了，可用于后续 **Step 3 交叉验证 + BMA** 以及帕累托分析。


# 定义流域信息

In [2]:
# 读取流域信息
basin_info      = pd.read_excel('../../Data/Basin_Selection/All_Selected_Basins.xlsx')
basin_list      = basin_info['stat_num']
cali_start_list = basin_info['cali_start']
cali_end_list   = basin_info['cali_end']
vali_start_list = basin_info['vali_start']
vali_end_list   = basin_info['vali_end']

# 定义数据读取函数

In [3]:
# 集总式模型数据读取
def get_data_lumped(basin, basin_idx):
    filepath = f"../../../2025_03_Hydrological_Models/Data/New_Hydro_Climatic/NHC_{basin}.txt"
    hc_data = pd.read_csv(filepath, sep = '\t', header=0, index_col='Time', parse_dates=['Time'])
    cali_start = pd.to_datetime(f"{str(cali_start_list[basin_idx])}-01-01")
    cali_end   = pd.to_datetime(f"{str(cali_end_list[basin_idx])}-12-31")
    vali_start = pd.to_datetime(f"{str(vali_start_list[basin_idx])}-01-01")
    vali_end   = pd.to_datetime(f"{str(vali_end_list[basin_idx])}-12-31")
    esim_start = pd.Timestamp('1980-01-01')
    esim_end   = pd.Timestamp('2022-12-31')

    cali_data = hc_data.loc[cali_start : cali_end]
    vali_data = hc_data.loc[vali_start : vali_end]
    esim_data = hc_data.loc[esim_start : esim_end]

    x_cali = cali_data[['PRE_CRU', 'TMP_CRU', 'PET_CRU']].values + 1e-6
    y_cali = cali_data['RUN'].values
    x_vali = vali_data[['PRE_CRU', 'TMP_CRU', 'PET_CRU']].values + 1e-6
    y_vali = vali_data['RUN'].values
    x_esim = esim_data[['PRE_CRU', 'TMP_CRU', 'PET_CRU']].values + 1e-6

    ae_filepath = f"../../../2025_03_Hydrological_Models/Data/AE/AE_{basin}.txt"
    ae_obs = pd.read_csv(ae_filepath, sep = '\t', header=0, index_col='Time', parse_dates=['Time'])

    y_esim = ae_obs['AE'].values
    return x_cali, y_cali, x_vali, y_vali, x_esim, y_esim

STP = pd.read_excel("../../../2025_03_Hydrological_Models/Raw_Data/D_TEXTURE_USDA.xlsx")
STP = STP[['CODE', 's', 'fc', 'wp']].to_numpy()

def read_georaster(filepath):
    with rasterio.open(filepath) as src:
        temp_data = src.read(1)
        new_data = temp_data.copy().astype(np.float64)
        new_data[new_data < 0] = np.nan
        # 获取经纬度范围
        bounds = src.bounds
        # 获取分辨率
        res = src.res
        # 生成经纬度坐标
        lon = np.round(np.arange(bounds.left + res[0] / 2, bounds.right, res[0]), 3)
        lat = np.round(np.arange(bounds.top - res[1] / 2, bounds.bottom, -res[1]), 3)
    return new_data, lon, lat, bounds, res[0]

def read_nc(filepath, varname):
    cf_info = Dataset(filepath)
    cf_data = np.rot90(np.flip(cf_info.variables[varname][:].data, axis=1), k = -1, axes=(1 ,2))
    cf_lon = cf_info.variables['lon'][:].data
    cf_lat = cf_info.variables['lat'][:].data
    cf_info.close()
    return cf_data, cf_lon, cf_lat

spec = [
    ('PRE',      float64[:, :, :]),
    ('TMP',      float64[:, :, :]),
    ('PET',      float64[:, :, :]),
    ('TI',       float64[:, :]),
    ('ST',       float64[:, :]),
    ('STP',      float64[:, :]),
    ('mask',     float64[:, :]),
    ('mask_res', float64),
    ('mask_lon', float64[:]),
    ('mask_lat', float64[:]),
    ('TI_lon',   float64[:]),
    ('TI_lat',   float64[:]),
]
@jitclass(spec)
class inputData:
    def __init__(self, PRE, TMP, PET, TI, ST, STP, mask, mask_res, mask_lon, mask_lat, TI_lon, TI_lat):
        self.PRE        = PRE
        self.TMP        = TMP
        self.PET        = PET
        self.TI         = TI
        self.ST         = ST
        self.STP        = STP
        self.mask       = mask
        self.mask_res   = mask_res
        self.mask_lon   = mask_lon
        self.mask_lat   = mask_lat
        self.TI_lon     = TI_lon
        self.TI_lat     = TI_lat
def get_data_distributed(basin, b):
    filepath    = f"../../../2025_03_Hydrological_Models/Data/New_Hydro_Climatic/NHC_{basin}.txt"
    # time for calibration and validation
    hc_data     = pd.read_csv(filepath, sep='\t', header=0, index_col='Time', parse_dates=['Time'])
    cali_start  = pd.to_datetime(f"{str(cali_start_list[b])}-01-01")
    cali_end    = pd.to_datetime(f"{str(cali_end_list[b])}-12-31")
    vali_start  = pd.to_datetime(f"{str(vali_start_list[b])}-01-01")
    vali_end    = pd.to_datetime(f"{str(vali_end_list[b])}-12-31")
    esim_start  = pd.Timestamp('1980-01-01')
    esim_end    = pd.Timestamp('2022-12-31')
    # time index
    cali_loc    = (hc_data.index >= cali_start) & (hc_data.index <= cali_end)
    vali_loc    = (hc_data.index >= vali_start) & (hc_data.index <= vali_end)
    esim_loc    = (hc_data.index >= esim_start) & (hc_data.index <= esim_end)

    _, _, _, _, _, y_esim = get_data_lumped(basin, b)
    # runoff series
    y_cali = hc_data.loc[cali_start:cali_end]['RUN'].to_numpy()
    y_vali = hc_data.loc[vali_start:vali_end]['RUN'].to_numpy()
    # basin grid
    basin_mask, mask_lon, mask_lat, mask_bounds, mask_res = read_georaster(f"../../../2025_03_Hydrological_Models/Data/Basin_Boundary_TIF/{basin}.tif")
    basin_mask[basin_mask >= 0] = 1
    # ti
    TI, TI_lon, TI_lat, TI_bounds, TI_res = read_georaster(f"../../../2025_03_Hydrological_Models/Data/Underlying/TI/TI_{basin}.tif")
    # soil_texture
    ST, ST_lon, ST_lat, ST_bounds, ST_res = read_georaster(f"../../../2025_03_Hydrological_Models/Data/Underlying/Soil_Texture/Soil_Texture_{basin}.tif")
    # climatic forcing
    PRE, cf_lon, cf_lat = read_nc(f"../../../2025_03_Hydrological_Models/Data/CRU/PRE_CRU/PRE_{basin}.nc", 'PRE')
    TMP, _, _ = read_nc(f"../../../2025_03_Hydrological_Models/Data/CRU/TMP_CRU/TMP_{basin}.nc", 'TMP')
    PET, _, _ = read_nc(f"../../../2025_03_Hydrological_Models/Data/CRU/PET_CRU/PET_{basin}.nc", 'PET')

    PRE_cali = PRE[cali_loc, :, :]
    TMP_cali = TMP[cali_loc, :, :]
    PET_cali = PET[cali_loc, :, :] * 30.4
    PRE_vali = PRE[vali_loc, :, :]
    TMP_vali = TMP[vali_loc, :, :]
    PET_vali = PET[vali_loc, :, :] * 30.4
    PRE_esim = PRE[esim_loc, :, :]
    TMP_esim = TMP[esim_loc, :, :]
    PET_esim = PET[esim_loc, :, :] * 30.4

    x_cali = inputData(PRE_cali, TMP_cali, PET_cali, TI, ST, STP, basin_mask, mask_res, mask_lon, mask_lat, TI_lon, TI_lat)
    x_vali = inputData(PRE_vali, TMP_vali, PET_vali, TI, ST, STP, basin_mask, mask_res, mask_lon, mask_lat, TI_lon, TI_lat)
    x_esim = inputData(PRE_esim, TMP_esim, PET_esim, TI, ST, STP, basin_mask, mask_res, mask_lon, mask_lat, TI_lon, TI_lat)
    return x_cali, y_cali, x_vali, y_vali, x_esim, y_esim

# 获取率定参数

## 估计box-cox的lambda值

In [ ]:
lambda_list = np.full(len(basin_list), np.nan)
for b in trange(len(basin_list)):
    basin = str(basin_list[b])
    x_cali_l, y_cali_l, _, _, x_esim_l, y_esim_l = get_data_lumped(basin, b)
    y_pos = y_cali_l[y_cali_l > 0] + 1e-5
    y_trans, fitted_lambda = boxcox(y_pos)
    lambda_list[b] = fitted_lambda
lambda_ave_r = np.nanmean(lambda_list)

for b in trange(len(basin_list)):
    basin = str(basin_list[b])
    x_cali_l, y_cali_l, _, _, x_esim_l, y_esim_l = get_data_lumped(basin, b)
    y_pos = y_esim_l[y_esim_l > 0] + 1e-5
    y_trans, fitted_lambda = boxcox(y_pos)
    lambda_list[b] = fitted_lambda
lambda_ave_e = np.nanmean(lambda_list)
print(f"Average lambda for runoff: {lambda_ave_r}")
print(f"Average lambda for AE: {lambda_ave_e}")

  0%|          | 0/2003 [00:00<?, ?it/s]

  0%|          | 0/2003 [00:00<?, ?it/s]

Average lambda for runoff: -0.08544487026765571
Average lambda for AE: 0.5207486965559343


# 径流模拟计算对数似然

In [5]:
def loglik_gaussian_with_mu(residuals):
    n = residuals.size
    mu_hat = np.nanmean(residuals)
    sigma2_hat = np.nanmean((residuals - mu_hat) ** 2)
    lnL = -0.5 * n * (np.log(2*np.pi*sigma2_hat) + 1)
    return lnL

def information_criteria(lnL: float, k: int, n: int):
    AIC = 2*k - 2*lnL
    BIC = k*np.log(n) - 2*lnL
    return AIC/n, BIC/n

# AIAC方法计算多模型平均

In [5]:
result_vars = ['r_lnL', 'r_ic', 'r_re', 'r_kge', 'e_lnL', 'e_ic', 'e_re', 'e_kge', 'r_w', 'e_w']
models = ['YM', 'AM', 'DM', 'GYM']
results = {f"{var}_{model}": np.full(2003, np.nan) for var in result_vars for model in models}
results['ic_min'] = np.full(2003, np.nan)
results['weighted_KGE'] = np.full(2003, np.nan)
results['weighted_RE']  = np.full(2003, np.nan)
results['weighted_check'] = np.full(2003, np.nan)

params_YM  = pd.read_csv("../../Data/Params/03_mYWBM_Best_Params_CF.txt", sep="\t", index_col='stat_num')
params_AM  = pd.read_csv("../../Data/Params/03_abcd_Best_Params_CF.txt", sep="\t", index_col='stat_num')
params_DM  = pd.read_csv("../../Data/Params/03_DWBM_Best_Params_CF.txt", sep="\t", index_col='stat_num')
params_GYM = pd.read_csv("../../Data/Params/03_GmYWBM_Best_Params_CF.txt", sep="\t", index_col='stat_num')

In [ ]:
def boxcox_transform(y, lam, eps=1e-5):
    # λ=1：原始值（不变换）
    # λ=0：对数变换
    y = y + eps  # 保证正数
    if lam == 0:
        return np.log(y)
    else:
        return (np.power(y, lam) - 1) / lam
lambda_ave_r = -0.08544487038241699
lambda_ave_e = 0.5207486965559343
for b in trange(len(basin_list)):
    basin = str(basin_list[b])
    # print(f"Working: {b+1}. {basin}")
    ## 流域数据读取
    # 径流模拟数据
    x_cali_l, y_cali_l, _, _, x_esim_l, y_esim_l = get_data_lumped(basin, b)
    x_cali_g, y_cali_g, _, _, x_esim_g, y_esim_g = get_data_distributed(basin, b)
    # 径流模拟
    r_sim_ym, _  = mYWBMnlS_RE(x_cali_l, params_YM.loc[basin].values)
    r_sim_am, _  = abcdnlS_RE(x_cali_l, params_AM.loc[basin].values)
    r_sim_dm, _  = DWBMnlS_RE(x_cali_l, params_DM.loc[basin].values)
    r_sim_gym, _ = GmYWBM_RE(x_cali_g, params_GYM.loc[basin].values)
    _, e_sim_ym  = mYWBMnlS_RE(x_esim_l, params_YM.loc[basin].values)
    _, e_sim_am  = abcdnlS_RE(x_esim_l, params_AM.loc[basin].values)
    _, e_sim_dm  = DWBMnlS_RE(x_esim_l, params_DM.loc[basin].values)
    _, e_sim_gym = GmYWBM_RE(x_esim_g, params_GYM.loc[basin].values)
    y_mask = ~np.isnan(y_cali_l) & ~np.isnan(r_sim_ym) & ~np.isnan(r_sim_am) & ~np.isnan(r_sim_dm)
    y_cali_l  = y_cali_l[y_mask]
    r_sim_ym  = r_sim_ym[y_mask]
    r_sim_am  = r_sim_am[y_mask]
    r_sim_dm  = r_sim_dm[y_mask]
    r_sim_gym = r_sim_gym[y_mask]
    y_mask = ~np.isnan(y_esim_l) & ~np.isnan(e_sim_ym) & ~np.isnan(e_sim_am) & ~np.isnan(e_sim_dm)
    y_esim_l  = y_esim_l[y_mask]
    e_sim_ym  = e_sim_ym[y_mask]
    e_sim_am  = e_sim_am[y_mask]
    e_sim_dm  = e_sim_dm[y_mask]
    e_sim_gym = e_sim_gym[y_mask]
    # 径流模拟残差
    y_trans = boxcox_transform(y_cali_l, lambda_ave_r)
    r_sim_ym_trans  = boxcox_transform(r_sim_ym, lambda_ave_r)
    r_sim_am_trans  = boxcox_transform(r_sim_am, lambda_ave_r)
    r_sim_dm_trans  = boxcox_transform(r_sim_dm, lambda_ave_r)
    r_sim_gym_trans = boxcox_transform(r_sim_gym, lambda_ave_r)
    r_res_ym  = r_sim_ym_trans  - y_trans
    r_res_am  = r_sim_am_trans  - y_trans
    r_res_dm  = r_sim_dm_trans  - y_trans
    r_res_gym = r_sim_gym_trans - y_trans
    e_trans = boxcox_transform(y_esim_l, lambda_ave_e)
    e_sim_ym_trans  = boxcox_transform(e_sim_ym, lambda_ave_e)
    e_sim_am_trans  = boxcox_transform(e_sim_am, lambda_ave_e)
    e_sim_dm_trans  = boxcox_transform(e_sim_dm, lambda_ave_e)
    e_sim_gym_trans = boxcox_transform(e_sim_gym, lambda_ave_e)
    e_res_ym  = e_sim_ym_trans  - e_trans
    e_res_am  = e_sim_am_trans  - e_trans
    e_res_dm  = e_sim_dm_trans  - e_trans
    e_res_gym = e_sim_gym_trans - e_trans
    ## 计算对数似然
    results['r_lnL_YM'][b]  = loglik_gaussian_with_mu(r_res_ym)
    results['r_lnL_AM'][b]  = loglik_gaussian_with_mu(r_res_am)
    results['r_lnL_DM'][b]  = loglik_gaussian_with_mu(r_res_dm)
    results['r_lnL_GYM'][b] = loglik_gaussian_with_mu(r_res_gym)
    results['e_lnL_YM'][b]  = loglik_gaussian_with_mu(e_res_ym)
    results['e_lnL_AM'][b]  = loglik_gaussian_with_mu(e_res_am)
    results['e_lnL_DM'][b]  = loglik_gaussian_with_mu(e_res_dm)
    results['e_lnL_GYM'][b] = loglik_gaussian_with_mu(e_res_gym)
    ## 计算信息准则
    n = len(np.where(~np.isnan(y_cali_l))[0])
    k = 7
    # 残差平方和
    r_sse_ym  = np.nansum(r_res_ym**2)
    r_sse_am  = np.nansum(r_res_am**2)
    r_sse_dm  = np.nansum(r_res_dm**2)
    r_sse_gym = np.nansum(r_res_gym**2)
    e_sse_ym  = np.nansum(e_res_ym**2)
    e_sse_am  = np.nansum(e_res_am**2)
    e_sse_dm  = np.nansum(e_res_dm**2)
    e_sse_gym = np.nansum(e_res_gym**2)
    # 信息准则
    results['r_ic_YM'][b]  = n * np.log(r_sse_ym / n)
    results['r_ic_AM'][b]  = n * np.log(r_sse_am / n)
    results['r_ic_DM'][b]  = n * np.log(r_sse_dm / n)
    results['r_ic_GYM'][b] = n * np.log(r_sse_gym / n)
    results['e_ic_YM'][b]  = n * np.log(e_sse_ym / n)
    results['e_ic_AM'][b]  = n * np.log(e_sse_am / n)
    results['e_ic_DM'][b]  = n * np.log(e_sse_dm / n)
    results['e_ic_GYM'][b] = n * np.log(e_sse_gym / n)
    ## 计算相对误差
    results['r_re_YM'][b]  = relative_error(y_trans, r_sim_ym_trans)
    results['r_re_AM'][b]  = relative_error(y_trans, r_sim_am_trans)
    results['r_re_DM'][b]  = relative_error(y_trans, r_sim_dm_trans)
    results['r_re_GYM'][b] = relative_error(y_trans, r_sim_gym_trans)
    results['e_re_YM'][b]  = relative_error(e_trans, e_sim_ym_trans)
    results['e_re_AM'][b]  = relative_error(e_trans, e_sim_am_trans)
    results['e_re_DM'][b]  = relative_error(e_trans, e_sim_dm_trans)
    results['e_re_GYM'][b] = relative_error(e_trans, e_sim_gym_trans)
    ## 计算KGE
    results['r_kge_YM'][b]  = nash_sutcliffe_efficiency(y_trans, r_sim_ym_trans)
    results['r_kge_AM'][b]  = nash_sutcliffe_efficiency(y_trans, r_sim_am_trans)
    results['r_kge_DM'][b]  = nash_sutcliffe_efficiency(y_trans, r_sim_dm_trans)
    results['r_kge_GYM'][b] = nash_sutcliffe_efficiency(y_trans, r_sim_gym_trans)
    results['e_kge_YM'][b]  = nash_sutcliffe_efficiency(e_trans, e_sim_ym_trans)
    results['e_kge_AM'][b]  = nash_sutcliffe_efficiency(e_trans, e_sim_am_trans)
    results['e_kge_DM'][b]  = nash_sutcliffe_efficiency(e_trans, e_sim_dm_trans)
    results['e_kge_GYM'][b] = nash_sutcliffe_efficiency(e_trans, e_sim_gym_trans)
    # 最小信息准则
    ic_min = np.nanmin([results['r_ic_YM'][b], results['r_ic_AM'][b], results['r_ic_DM'][b], results['r_ic_GYM'][b]])
    delta_ic_ym  = results['r_ic_YM'][b] - ic_min
    delta_ic_am  = results['r_ic_AM'][b] - ic_min
    delta_ic_dm  = results['r_ic_DM'][b] - ic_min
    delta_ic_gym = results['r_ic_GYM'][b] - ic_min
    # 计算AIC权重
    w_ic_ym  = np.exp(-0.5 * delta_ic_ym) / (np.exp(-0.5 * delta_ic_ym) + np.exp(-0.5 * delta_ic_am) + np.exp(-0.5 * delta_ic_dm) + np.exp(-0.5 * delta_ic_gym))
    w_ic_am  = np.exp(-0.5 * delta_ic_am) / (np.exp(-0.5 * delta_ic_ym) + np.exp(-0.5 * delta_ic_am) + np.exp(-0.5 * delta_ic_dm) + np.exp(-0.5 * delta_ic_gym))
    w_ic_dm  = np.exp(-0.5 * delta_ic_dm) / (np.exp(-0.5 * delta_ic_ym) + np.exp(-0.5 * delta_ic_am) + np.exp(-0.5 * delta_ic_dm) + np.exp(-0.5 * delta_ic_gym))
    w_ic_gym = np.exp(-0.5 * delta_ic_gym) / (np.exp(-0.5 * delta_ic_ym) + np.exp(-0.5 * delta_ic_am) + np.exp(-0.5 * delta_ic_dm) + np.exp(-0.5 * delta_ic_gym))
    results['r_w_YM'][b]  = w_ic_ym
    results['r_w_AM'][b]  = w_ic_am
    results['r_w_DM'][b]  = w_ic_dm
    results['r_w_GYM'][b] = w_ic_gym
    # 最小信息准则
    ic_min = np.nanmin([results['e_ic_YM'][b], results['e_ic_AM'][b], results['e_ic_DM'][b], results['e_ic_GYM'][b]])
    delta_ic_ym  = results['e_ic_YM'][b] - ic_min
    delta_ic_am  = results['e_ic_AM'][b] - ic_min
    delta_ic_dm  = results['e_ic_DM'][b] - ic_min
    delta_ic_gym = results['e_ic_GYM'][b] - ic_min
    # 计算AIC权重
    w_ic_ym  = np.exp(-0.5 * delta_ic_ym) / (np.exp(-0.5 * delta_ic_ym) + np.exp(-0.5 * delta_ic_am) + np.exp(-0.5 * delta_ic_dm) + np.exp(-0.5 * delta_ic_gym))
    w_ic_am  = np.exp(-0.5 * delta_ic_am) / (np.exp(-0.5 * delta_ic_ym) + np.exp(-0.5 * delta_ic_am) + np.exp(-0.5 * delta_ic_dm) + np.exp(-0.5 * delta_ic_gym))
    w_ic_dm  = np.exp(-0.5 * delta_ic_dm) / (np.exp(-0.5 * delta_ic_ym) + np.exp(-0.5 * delta_ic_am) + np.exp(-0.5 * delta_ic_dm) + np.exp(-0.5 * delta_ic_gym))
    w_ic_gym = np.exp(-0.5 * delta_ic_gym) / (np.exp(-0.5 * delta_ic_ym) + np.exp(-0.5 * delta_ic_am) + np.exp(-0.5 * delta_ic_dm) + np.exp(-0.5 * delta_ic_gym))
    results['e_w_YM'][b]  = w_ic_ym
    results['e_w_AM'][b]  = w_ic_am
    results['e_w_DM'][b]  = w_ic_dm
    results['e_w_GYM'][b] = w_ic_gym
# 保存结果
df_results = pd.DataFrame(results, index=basin_list)
df_results.index.name = 'stat_num'
df_results.to_csv("../../Results/Weighted_Average/Weighted_Average_Results_AIAC.txt", sep = '\t', header=True, float_format="%.3f", index=True)

In [6]:
result_vars = ['r_lnL', 'r_ic', 'r_re', 'r_kge', 'e_lnL', 'e_ic', 'e_re', 'e_kge', 'r_w', 'e_w']
models = ['YM', 'AM', 'DM']
results = {f"{var}_{model}": np.full(2003, np.nan) for var in result_vars for model in models}
results['ic_min'] = np.full(2003, np.nan)
results['weighted_KGE'] = np.full(2003, np.nan)
results['weighted_RE']  = np.full(2003, np.nan)
results['weighted_check'] = np.full(2003, np.nan)

params_YM  = pd.read_csv("../../Data/Params/03_mYWBM_Best_Params_CF.txt", sep="\t", index_col='stat_num')
params_AM  = pd.read_csv("../../Data/Params/03_abcd_Best_Params_CF.txt", sep="\t", index_col='stat_num')
params_DM  = pd.read_csv("../../Data/Params/03_DWBM_Best_Params_CF.txt", sep="\t", index_col='stat_num')
params_GYM = pd.read_csv("../../Data/Params/03_GmYWBM_Best_Params_CF.txt", sep="\t", index_col='stat_num')

In [7]:
def boxcox_transform(y, lam, eps=1e-5):
    # λ=1：原始值（不变换）
    # λ=0：对数变换
    y = y + eps  # 保证正数
    if lam == 0:
        return np.log(y)
    else:
        return (np.power(y, lam) - 1) / lam
lambda_ave_r = -0.08544487038241699
lambda_ave_e = 0.5207486965559343
for b in trange(len(basin_list)):
    basin = str(basin_list[b])
    # print(f"Working: {b+1}. {basin}")
    ## 流域数据读取
    # 径流模拟数据
    x_cali_l, y_cali_l, _, _, x_esim_l, y_esim_l = get_data_lumped(basin, b)
    y_cali_l[y_cali_l < 0] = 0
    y_esim_l[y_esim_l < 0] = 0
    y_pos_cali = y_cali_l[y_cali_l > 0] + 1e-5
    y_trans, fitted_lambda_cali = boxcox(y_pos_cali)
    y_pos_esim = y_esim_l[y_esim_l > 0] + 1e-5
    y_trans, fitted_lambda_esim = boxcox(y_pos_esim)

    # 径流模拟
    r_sim_ym, _  = mYWBMnlS_RE(x_cali_l + 1e-5, params_YM.loc[basin].values + 1e-5)
    r_sim_am, _  = abcdnlS_RE(x_cali_l + 1e-5, params_AM.loc[basin].values + 1e-5)
    r_sim_dm, _  = DWBMnlS_RE(x_cali_l + 1e-5, params_DM.loc[basin].values + 1e-5)
    _, e_sim_ym  = mYWBMnlS_RE(x_esim_l + 1e-5, params_YM.loc[basin].values + 1e-5)
    _, e_sim_am  = abcdnlS_RE(x_esim_l + 1e-5, params_AM.loc[basin].values + 1e-5)
    _, e_sim_dm  = DWBMnlS_RE(x_esim_l + 1e-5, params_DM.loc[basin].values + 1e-5)

    y_mask = ~np.isnan(r_sim_ym) & ~np.isnan(r_sim_am) & ~np.isnan(r_sim_dm) & ~np.isnan(y_cali_l)
    y_cali_l  = y_cali_l[y_mask]
    r_sim_ym  = r_sim_ym[y_mask]
    r_sim_am  = r_sim_am[y_mask]
    r_sim_dm  = r_sim_dm[y_mask]

    y_mask = ~np.isnan(e_sim_ym) & ~np.isnan(e_sim_am) & ~np.isnan(e_sim_dm) & ~np.isnan(y_esim_l)
    y_esim_l  = y_esim_l[y_mask]
    e_sim_ym  = e_sim_ym[y_mask]
    e_sim_am  = e_sim_am[y_mask]
    e_sim_dm  = e_sim_dm[y_mask]

    # 径流模拟残差
    y_trans = boxcox_transform(y_cali_l, fitted_lambda_cali)
    r_sim_ym_trans  = boxcox_transform(r_sim_ym, fitted_lambda_cali)
    r_sim_am_trans  = boxcox_transform(r_sim_am, fitted_lambda_cali)
    r_sim_dm_trans  = boxcox_transform(r_sim_dm, fitted_lambda_cali)
    r_res_ym  = r_sim_ym_trans  - y_trans
    r_res_am  = r_sim_am_trans  - y_trans
    r_res_dm  = r_sim_dm_trans  - y_trans

    e_trans = boxcox_transform(y_esim_l, fitted_lambda_esim)
    e_sim_ym_trans  = boxcox_transform(e_sim_ym, fitted_lambda_esim)
    e_sim_am_trans  = boxcox_transform(e_sim_am, fitted_lambda_esim)
    e_sim_dm_trans  = boxcox_transform(e_sim_dm, fitted_lambda_esim)
    e_res_ym  = e_sim_ym_trans  - e_trans
    e_res_am  = e_sim_am_trans  - e_trans
    e_res_dm  = e_sim_dm_trans  - e_trans
    ## 计算对数似然
    results['r_lnL_YM'][b]  = loglik_gaussian_with_mu(r_res_ym)
    results['r_lnL_AM'][b]  = loglik_gaussian_with_mu(r_res_am)
    results['r_lnL_DM'][b]  = loglik_gaussian_with_mu(r_res_dm)
    results['e_lnL_YM'][b]  = loglik_gaussian_with_mu(e_res_ym)
    results['e_lnL_AM'][b]  = loglik_gaussian_with_mu(e_res_am)
    results['e_lnL_DM'][b]  = loglik_gaussian_with_mu(e_res_dm)
    ## 计算信息准则
    n = len(np.where(~np.isnan(y_cali_l))[0])
    k = 7
    # 残差平方和
    r_sse_ym  = np.nansum(r_res_ym**2)
    r_sse_am  = np.nansum(r_res_am**2)
    r_sse_dm  = np.nansum(r_res_dm**2)

    e_sse_ym  = np.nansum(e_res_ym**2)
    e_sse_am  = np.nansum(e_res_am**2)
    e_sse_dm  = np.nansum(e_res_dm**2)

    # 信息准则
    results['r_ic_YM'][b]  = n * np.log(r_sse_ym / n)
    results['r_ic_AM'][b]  = n * np.log(r_sse_am / n)
    results['r_ic_DM'][b]  = n * np.log(r_sse_dm / n)

    results['e_ic_YM'][b]  = n * np.log(e_sse_ym / n)
    results['e_ic_AM'][b]  = n * np.log(e_sse_am / n)
    results['e_ic_DM'][b]  = n * np.log(e_sse_dm / n)

    ## 计算相对误差
    results['r_re_YM'][b]  = relative_error(y_trans, r_sim_ym_trans)
    results['r_re_AM'][b]  = relative_error(y_trans, r_sim_am_trans)
    results['r_re_DM'][b]  = relative_error(y_trans, r_sim_dm_trans)

    results['e_re_YM'][b]  = relative_error(e_trans, e_sim_ym_trans)
    results['e_re_AM'][b]  = relative_error(e_trans, e_sim_am_trans)
    results['e_re_DM'][b]  = relative_error(e_trans, e_sim_dm_trans)

    ## 计算KGE
    results['r_kge_YM'][b]  = nash_sutcliffe_efficiency(y_trans, r_sim_ym_trans)
    results['r_kge_AM'][b]  = nash_sutcliffe_efficiency(y_trans, r_sim_am_trans)
    results['r_kge_DM'][b]  = nash_sutcliffe_efficiency(y_trans, r_sim_dm_trans)

    results['e_kge_YM'][b]  = nash_sutcliffe_efficiency(e_trans, e_sim_ym_trans)
    results['e_kge_AM'][b]  = nash_sutcliffe_efficiency(e_trans, e_sim_am_trans)
    results['e_kge_DM'][b]  = nash_sutcliffe_efficiency(e_trans, e_sim_dm_trans)

    # 最小信息准则
    ic_min = np.nanmin([results['r_ic_YM'][b], results['r_ic_AM'][b], results['r_ic_DM'][b]])
    delta_ic_ym  = results['r_ic_YM'][b] - ic_min
    delta_ic_am  = results['r_ic_AM'][b] - ic_min
    delta_ic_dm  = results['r_ic_DM'][b] - ic_min

    # 计算AIC权重
    w_ic_ym  = np.exp(-0.5 * delta_ic_ym) / (np.exp(-0.5 * delta_ic_ym) + np.exp(-0.5 * delta_ic_am) + np.exp(-0.5 * delta_ic_dm))
    w_ic_am  = np.exp(-0.5 * delta_ic_am) / (np.exp(-0.5 * delta_ic_ym) + np.exp(-0.5 * delta_ic_am) + np.exp(-0.5 * delta_ic_dm))
    w_ic_dm  = np.exp(-0.5 * delta_ic_dm) / (np.exp(-0.5 * delta_ic_ym) + np.exp(-0.5 * delta_ic_am) + np.exp(-0.5 * delta_ic_dm))

    results['r_w_YM'][b]  = w_ic_ym
    results['r_w_AM'][b]  = w_ic_am
    results['r_w_DM'][b]  = w_ic_dm

    # 最小信息准则
    ic_min = np.nanmin([results['e_ic_YM'][b], results['e_ic_AM'][b], results['e_ic_DM'][b]])
    delta_ic_ym  = results['e_ic_YM'][b] - ic_min
    delta_ic_am  = results['e_ic_AM'][b] - ic_min
    delta_ic_dm  = results['e_ic_DM'][b] - ic_min

    # 计算AIC权重
    w_ic_ym  = np.exp(-0.5 * delta_ic_ym) / (np.exp(-0.5 * delta_ic_ym) + np.exp(-0.5 * delta_ic_am) + np.exp(-0.5 * delta_ic_dm))
    w_ic_am  = np.exp(-0.5 * delta_ic_am) / (np.exp(-0.5 * delta_ic_ym) + np.exp(-0.5 * delta_ic_am) + np.exp(-0.5 * delta_ic_dm))
    w_ic_dm  = np.exp(-0.5 * delta_ic_dm) / (np.exp(-0.5 * delta_ic_ym) + np.exp(-0.5 * delta_ic_am) + np.exp(-0.5 * delta_ic_dm))

    results['e_w_YM'][b]  = w_ic_ym
    results['e_w_AM'][b]  = w_ic_am
    results['e_w_DM'][b]  = w_ic_dm

# 保存结果
df_results = pd.DataFrame(results, index=basin_list)
df_results.index.name = 'stat_num'
df_results.to_csv("../../Results/Weighted_Average/Weighted_Average_Results_AIAC_3M.txt", sep = '\t', header=True, float_format="%.3f", index=True)

  0%|          | 0/2003 [00:00<?, ?it/s]

# BMA方法计算多模型平均

In [8]:
from scipy.stats import norm
def fit_bma_em(y, X, max_iter=200, tol=1e-6, verbose=False):
    y = np.asarray(y).ravel()
    T = len(y)
    X = np.asarray(X)
    if X.shape[0] != T:
        raise ValueError("X and y must have the same number of rows")
    T, M = X.shape
    w = np.ones(M) / M
    a = np.zeros(M)
    b = np.ones(M)
    sigma = np.ones(M) * max(np.std(y), 1e-3)
    loglik_trace = []
    for iteration in range(max_iter):
        comp_means = (a[:, None] + b[:, None] * X.T).T
        pdf = np.empty((T, M))
        for i in range(M):
            pdf[:, i] = norm.pdf(y, loc=comp_means[:, i], scale=sigma[i] + 1e-12)
        weighted_pdf = pdf * w[np.newaxis, :]
        denom = np.sum(weighted_pdf, axis=1, keepdims=True)
        denom = np.maximum(denom, 1e-300)
        z = weighted_pdf / denom
        loglik = np.sum(np.log(denom.flatten()))
        loglik_trace.append(loglik)
        w_new = np.sum(z, axis=0) / T
        a_new, b_new, sigma_new = np.zeros_like(a), np.zeros_like(b), np.zeros_like(sigma)
        for i in range(M):
            wi = z[:, i]
            W_sum = np.sum(wi)
            if W_sum < 1e-12:
                a_new[i], b_new[i], sigma_new[i] = a[i], b[i], sigma[i]
                continue
            x_i = X[:, i]
            x_bar = np.sum(wi * x_i) / W_sum
            y_bar = np.sum(wi * y) / W_sum
            S_xy = np.sum(wi * (x_i - x_bar) * (y - y_bar))
            S_xx = np.sum(wi * (x_i - x_bar)**2)
            b_hat = S_xy / S_xx if S_xx != 0 else 0
            a_hat = y_bar - b_hat * x_bar
            resid = y - (a_hat + b_hat * x_i)
            sigma2 = np.sum(wi * resid**2) / W_sum
            sigma2 = max(sigma2, 1e-8)
            a_new[i], b_new[i], sigma_new[i] = a_hat, b_hat, np.sqrt(sigma2)
        w, a, b, sigma = w_new, a_new, b_new, sigma_new
        if iteration > 0 and abs(loglik_trace[-1] - loglik_trace[-2]) < tol:
            if verbose:
                print("Converged at iteration", iteration)
            break
    comp_means = (a[:, None] + b[:, None] * X.T).T
    pred_mean = np.sum(w[np.newaxis, :] * comp_means, axis=1)
    pred_second = np.sum(w[np.newaxis, :] * (sigma**2 + comp_means**2), axis=1)
    pred_var = np.maximum(pred_second - pred_mean**2, 0.0)
    return {"weights": w, "a": a, "b": b, "sigma": sigma, "pred_mean": pred_mean, "pred_var": pred_var}

In [10]:
result_vars = ['r_w', 'e_w']
models = ['YM', 'AM', 'DM', 'GYM']
results = {f"{var}_{model}": np.full(2003, np.nan) for var in result_vars for model in models}

for b in trange(len(basin_list)):
    basin = str(basin_list[b])
    # print(f"Working: {b+1}. {basin}")
    ## 流域数据读取
    # 径流模拟数据
    x_cali_l, y_cali_l, _, _, x_esim_l, y_esim_l = get_data_lumped(basin, b)
    x_cali_g, y_cali_g, _, _, x_esim_g, y_esim_g = get_data_distributed(basin, b)
    # 径流模拟
    r_sim_ym, _  = mYWBMnlS_RE(x_cali_l, params_YM.loc[basin].values)
    r_sim_am, _  = abcdnlS_RE(x_cali_l, params_AM.loc[basin].values)
    r_sim_dm, _  = DWBMnlS_RE(x_cali_l, params_DM.loc[basin].values)
    r_sim_gym, _ = GmYWBM_RE(x_cali_g, params_GYM.loc[basin].values)
    _, e_sim_ym  = mYWBMnlS_RE(x_esim_l, params_YM.loc[basin].values)
    _, e_sim_am  = abcdnlS_RE(x_esim_l, params_AM.loc[basin].values)
    _, e_sim_dm  = DWBMnlS_RE(x_esim_l, params_DM.loc[basin].values)
    _, e_sim_gym = GmYWBM_RE(x_esim_g, params_GYM.loc[basin].values)
    y_mask = ~np.isnan(y_cali_l) & ~np.isnan(r_sim_ym) & ~np.isnan(r_sim_am) & ~np.isnan(r_sim_dm)
    y_cali_l  = y_cali_l[y_mask]
    r_sim_ym  = r_sim_ym[y_mask]
    r_sim_am  = r_sim_am[y_mask]
    r_sim_dm  = r_sim_dm[y_mask]
    r_sim_gym = r_sim_gym[y_mask]
    y_mask = ~np.isnan(y_esim_l) & ~np.isnan(e_sim_ym) & ~np.isnan(e_sim_am) & ~np.isnan(e_sim_dm)
    y_esim_l  = y_esim_l[y_mask]
    e_sim_ym  = e_sim_ym[y_mask]
    e_sim_am  = e_sim_am[y_mask]
    e_sim_dm  = e_sim_dm[y_mask]
    e_sim_gym = e_sim_gym[y_mask]

    Rsim = np.vstack([r_sim_ym, r_sim_am, r_sim_dm, r_sim_gym]).T
    Esim = np.vstack([e_sim_ym, e_sim_am, e_sim_dm, e_sim_gym]).T

    Robs = y_cali_l
    res = fit_bma_em(Robs, Rsim, max_iter=500, tol=1e-7, verbose=False)
    w_ic_ym  = res['weights'][0]
    w_ic_am  = res['weights'][1]
    w_ic_dm  = res['weights'][2]
    w_ic_gym = res['weights'][3]
    results['r_w_YM'][b]  = w_ic_ym
    results['r_w_AM'][b]  = w_ic_am
    results['r_w_DM'][b]  = w_ic_dm
    results['r_w_GYM'][b] = w_ic_gym

    Eobs = y_esim_l
    res = fit_bma_em(Eobs, Esim, max_iter=500, tol=1e-7, verbose=False)
    w_ic_ym  = res['weights'][0]
    w_ic_am  = res['weights'][1]
    w_ic_dm  = res['weights'][2]
    w_ic_gym = res['weights'][3]
    results['e_w_YM'][b]  = w_ic_ym
    results['e_w_AM'][b]  = w_ic_am
    results['e_w_DM'][b]  = w_ic_dm
    results['e_w_GYM'][b] = w_ic_gym
df_results = pd.DataFrame(results, index=basin_list)
df_results.index.name = 'stat_num'
df_results.to_csv("../../Results/Weighted_Average/Weighted_Average_Results_BMA.txt", sep = '\t', header=True, float_format="%.3f", index=True)


  0%|          | 0/2003 [00:00<?, ?it/s]

In [11]:
result_vars = ['r_w', 'e_w']
models = ['YM', 'AM', 'DM']
results = {f"{var}_{model}": np.full(2003, np.nan) for var in result_vars for model in models}

for b in trange(len(basin_list)):
    basin = str(basin_list[b])
    # print(f"Working: {b+1}. {basin}")
    ## 流域数据读取
    # 径流模拟数据
    x_cali_l, y_cali_l, _, _, x_esim_l, y_esim_l = get_data_lumped(basin, b)
    # 径流模拟
    r_sim_ym, _  = mYWBMnlS_RE(x_cali_l, params_YM.loc[basin].values)
    r_sim_am, _  = abcdnlS_RE(x_cali_l, params_AM.loc[basin].values)
    r_sim_dm, _  = DWBMnlS_RE(x_cali_l, params_DM.loc[basin].values)

    _, e_sim_ym  = mYWBMnlS_RE(x_esim_l, params_YM.loc[basin].values)
    _, e_sim_am  = abcdnlS_RE(x_esim_l, params_AM.loc[basin].values)
    _, e_sim_dm  = DWBMnlS_RE(x_esim_l, params_DM.loc[basin].values)

    y_mask = ~np.isnan(y_cali_l) & ~np.isnan(r_sim_ym) & ~np.isnan(r_sim_am) & ~np.isnan(r_sim_dm)
    y_cali_l  = y_cali_l[y_mask]
    r_sim_ym  = r_sim_ym[y_mask]
    r_sim_am  = r_sim_am[y_mask]
    r_sim_dm  = r_sim_dm[y_mask]

    y_mask = ~np.isnan(y_esim_l) & ~np.isnan(e_sim_ym) & ~np.isnan(e_sim_am) & ~np.isnan(e_sim_dm)
    y_esim_l  = y_esim_l[y_mask]
    e_sim_ym  = e_sim_ym[y_mask]
    e_sim_am  = e_sim_am[y_mask]
    e_sim_dm  = e_sim_dm[y_mask]

    Rsim = np.vstack([r_sim_ym, r_sim_am, r_sim_dm]).T
    Esim = np.vstack([e_sim_ym, e_sim_am, e_sim_dm]).T

    Robs = y_cali_l
    res = fit_bma_em(Robs, Rsim, max_iter=500, tol=1e-7, verbose=False)
    w_ic_ym  = res['weights'][0]
    w_ic_am  = res['weights'][1]
    w_ic_dm  = res['weights'][2]

    results['r_w_YM'][b]  = w_ic_ym
    results['r_w_AM'][b]  = w_ic_am
    results['r_w_DM'][b]  = w_ic_dm


    Eobs = y_esim_l
    res = fit_bma_em(Eobs, Esim, max_iter=500, tol=1e-7, verbose=False)
    w_ic_ym  = res['weights'][0]
    w_ic_am  = res['weights'][1]
    w_ic_dm  = res['weights'][2]

    results['e_w_YM'][b]  = w_ic_ym
    results['e_w_AM'][b]  = w_ic_am
    results['e_w_DM'][b]  = w_ic_dm

df_results = pd.DataFrame(results, index=basin_list)
df_results.index.name = 'stat_num'
df_results.to_csv("../../Results/Weighted_Average/Weighted_Average_Results_BMA_3M.txt", sep = '\t', header=True, float_format="%.3f", index=True)


  0%|          | 0/2003 [00:00<?, ?it/s]

# GRC方法计算多模型平均

In [12]:
from scipy.optimize import minimize
def grc_weights(X, y, ridge=1e-8):
    T, M = X.shape
    def obj(w):
        preds = X.dot(w)
        return np.sum((y - preds)**2) + ridge * np.sum(w**2)
    w0 = np.ones(M) / M
    cons = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = [(0, 1) for _ in range(M)]
    res = minimize(obj, w0, bounds=bounds, constraints=cons, method='SLSQP', options={'ftol':1e-12, 'maxiter':1000})
    if not res.success:
        w = np.copy(w0)
        lr = 0.01
        for it in range(10000):
            grad = -2 * X.T.dot(y - X.dot(w)) + 2 * ridge * w
            w -= lr * grad
            w = np.maximum(w, 0)
            s = np.sum(w)
            if s == 0:
                w = np.ones(M) / M
            else:
                w = w / s
            if np.linalg.norm(grad) * lr < 1e-9:
                break
        return w, obj(w)
    return res.x, res.fun

In [13]:
result_vars = ['r_w', 'e_w']
models = ['YM', 'AM', 'DM', 'GYM']
results = {f"{var}_{model}": np.full(2003, np.nan) for var in result_vars for model in models}

for b in trange(len(basin_list)):
    basin = str(basin_list[b])
    # print(f"Working: {b+1}. {basin}")
    ## 流域数据读取
    # 径流模拟数据
    x_cali_l, y_cali_l, _, _, x_esim_l, y_esim_l = get_data_lumped(basin, b)
    x_cali_g, y_cali_g, _, _, x_esim_g, y_esim_g = get_data_distributed(basin, b)
    # 径流模拟
    r_sim_ym, _  = mYWBMnlS_RE(x_cali_l, params_YM.loc[basin].values)
    r_sim_am, _  = abcdnlS_RE(x_cali_l, params_AM.loc[basin].values)
    r_sim_dm, _  = DWBMnlS_RE(x_cali_l, params_DM.loc[basin].values)
    r_sim_gym, _ = GmYWBM_RE(x_cali_g, params_GYM.loc[basin].values)
    _, e_sim_ym  = mYWBMnlS_RE(x_esim_l, params_YM.loc[basin].values)
    _, e_sim_am  = abcdnlS_RE(x_esim_l, params_AM.loc[basin].values)
    _, e_sim_dm  = DWBMnlS_RE(x_esim_l, params_DM.loc[basin].values)
    _, e_sim_gym = GmYWBM_RE(x_esim_g, params_GYM.loc[basin].values)
    y_mask = ~np.isnan(y_cali_l) & ~np.isnan(r_sim_ym) & ~np.isnan(r_sim_am) & ~np.isnan(r_sim_dm)
    y_cali_l  = y_cali_l[y_mask]
    r_sim_ym  = r_sim_ym[y_mask]
    r_sim_am  = r_sim_am[y_mask]
    r_sim_dm  = r_sim_dm[y_mask]
    r_sim_gym = r_sim_gym[y_mask]
    y_mask = ~np.isnan(y_esim_l) & ~np.isnan(e_sim_ym) & ~np.isnan(e_sim_am) & ~np.isnan(e_sim_dm)
    y_esim_l  = y_esim_l[y_mask]
    e_sim_ym  = e_sim_ym[y_mask]
    e_sim_am  = e_sim_am[y_mask]
    e_sim_dm  = e_sim_dm[y_mask]
    e_sim_gym = e_sim_gym[y_mask]

    Rsim = np.vstack([r_sim_ym, r_sim_am, r_sim_dm, r_sim_gym]).T
    Esim = np.vstack([e_sim_ym, e_sim_am, e_sim_dm, e_sim_gym]).T

    Robs = y_cali_l
    weights_grc, objval = grc_weights(Rsim, Robs)
    R_grc = Rsim.dot(weights_grc)
    model_names = ['YM','AM','DM','GYM']
    weight_grc = dict(zip(model_names, weights_grc.tolist()))
    w_ic_ym  = weight_grc['YM']
    w_ic_am  = weight_grc['AM']
    w_ic_dm  = weight_grc['DM']
    w_ic_gym = weight_grc['GYM']
    results['r_w_YM'][b]  = w_ic_ym
    results['r_w_AM'][b]  = w_ic_am
    results['r_w_DM'][b]  = w_ic_dm
    results['r_w_GYM'][b] = w_ic_gym

    Eobs = y_esim_l
    weights_grc, objval = grc_weights(Esim, Eobs)
    E_grc = Esim.dot(weights_grc)
    model_names = ['YM','AM','DM','GYM']
    weight_grc = dict(zip(model_names, weights_grc.tolist()))
    w_ic_ym  = weight_grc['YM']
    w_ic_am  = weight_grc['AM']
    w_ic_dm  = weight_grc['DM']
    w_ic_gym = weight_grc['GYM']
    results['e_w_YM'][b]  = w_ic_ym
    results['e_w_AM'][b]  = w_ic_am
    results['e_w_DM'][b]  = w_ic_dm
    results['e_w_GYM'][b] = w_ic_gym
df_results = pd.DataFrame(results, index=basin_list)
df_results.index.name = 'stat_num'
df_results.to_csv("../../Results/Weighted_Average/Weighted_Average_Results_GRC.txt", sep = '\t', header=True, float_format="%.3f", index=True)


  0%|          | 0/2003 [00:00<?, ?it/s]

In [14]:
result_vars = ['r_w', 'e_w']
models = ['YM', 'AM', 'DM']
results = {f"{var}_{model}": np.full(2003, np.nan) for var in result_vars for model in models}

for b in trange(len(basin_list)):
    basin = str(basin_list[b])
    # print(f"Working: {b+1}. {basin}")
    ## 流域数据读取
    # 径流模拟数据
    x_cali_l, y_cali_l, _, _, x_esim_l, y_esim_l = get_data_lumped(basin, b)

    # 径流模拟
    r_sim_ym, _  = mYWBMnlS_RE(x_cali_l, params_YM.loc[basin].values)
    r_sim_am, _  = abcdnlS_RE(x_cali_l, params_AM.loc[basin].values)
    r_sim_dm, _  = DWBMnlS_RE(x_cali_l, params_DM.loc[basin].values)

    _, e_sim_ym  = mYWBMnlS_RE(x_esim_l, params_YM.loc[basin].values)
    _, e_sim_am  = abcdnlS_RE(x_esim_l, params_AM.loc[basin].values)
    _, e_sim_dm  = DWBMnlS_RE(x_esim_l, params_DM.loc[basin].values)

    y_mask = ~np.isnan(y_cali_l) & ~np.isnan(r_sim_ym) & ~np.isnan(r_sim_am) & ~np.isnan(r_sim_dm)
    y_cali_l  = y_cali_l[y_mask]
    r_sim_ym  = r_sim_ym[y_mask]
    r_sim_am  = r_sim_am[y_mask]
    r_sim_dm  = r_sim_dm[y_mask]

    y_mask = ~np.isnan(y_esim_l) & ~np.isnan(e_sim_ym) & ~np.isnan(e_sim_am) & ~np.isnan(e_sim_dm)
    y_esim_l  = y_esim_l[y_mask]
    e_sim_ym  = e_sim_ym[y_mask]
    e_sim_am  = e_sim_am[y_mask]
    e_sim_dm  = e_sim_dm[y_mask]


    Rsim = np.vstack([r_sim_ym, r_sim_am, r_sim_dm]).T
    Esim = np.vstack([e_sim_ym, e_sim_am, e_sim_dm]).T

    Robs = y_cali_l
    weights_grc, objval = grc_weights(Rsim, Robs)
    R_grc = Rsim.dot(weights_grc)
    model_names = ['YM','AM','DM']
    weight_grc = dict(zip(model_names, weights_grc.tolist()))
    w_ic_ym  = weight_grc['YM']
    w_ic_am  = weight_grc['AM']
    w_ic_dm  = weight_grc['DM']

    results['r_w_YM'][b]  = w_ic_ym
    results['r_w_AM'][b]  = w_ic_am
    results['r_w_DM'][b]  = w_ic_dm


    Eobs = y_esim_l
    weights_grc, objval = grc_weights(Esim, Eobs)
    E_grc = Esim.dot(weights_grc)
    model_names = ['YM','AM','DM']
    weight_grc = dict(zip(model_names, weights_grc.tolist()))
    w_ic_ym  = weight_grc['YM']
    w_ic_am  = weight_grc['AM']
    w_ic_dm  = weight_grc['DM']

    results['e_w_YM'][b]  = w_ic_ym
    results['e_w_AM'][b]  = w_ic_am
    results['e_w_DM'][b]  = w_ic_dm

df_results = pd.DataFrame(results, index=basin_list)
df_results.index.name = 'stat_num'
df_results.to_csv("../../Results/Weighted_Average/Weighted_Average_Results_GRC_3M.txt", sep = '\t', header=True, float_format="%.3f", index=True)


  0%|          | 0/2003 [00:00<?, ?it/s]

# GRB方法计算多模型平均

In [15]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

def grb_weights(X, y):
    # 确保输入是 numpy 数组
    X = np.asarray(X)
    y = np.asarray(y)
    
    # 如果 y 是一维的，转为列向量
    if y.ndim == 1:
        y = y.reshape(-1, 1)
    
    # 最小二乘估计
    XtX = X.T @ X
    Xty = X.T @ y
    weights = np.linalg.pinv(XtX) @ Xty  # 用伪逆更稳健
    
    # 转为行向量
    weights = weights.flatten()
    
    return weights

In [16]:
result_vars = ['r_w', 'e_w']
models = ['YM', 'AM', 'DM', 'GYM']
results = {f"{var}_{model}": np.full(2003, np.nan) for var in result_vars for model in models}

for b in trange(len(basin_list)):
    basin = str(basin_list[b])
    # print(f"Working: {b+1}. {basin}")
    ## 流域数据读取
    # 径流模拟数据
    x_cali_l, y_cali_l, _, _, x_esim_l, y_esim_l = get_data_lumped(basin, b)
    x_cali_g, y_cali_g, _, _, x_esim_g, y_esim_g = get_data_distributed(basin, b)
    # 径流模拟
    r_sim_ym, _  = mYWBMnlS_RE(x_cali_l, params_YM.loc[basin].values)
    r_sim_am, _  = abcdnlS_RE(x_cali_l, params_AM.loc[basin].values)
    r_sim_dm, _  = DWBMnlS_RE(x_cali_l, params_DM.loc[basin].values)
    r_sim_gym, _ = GmYWBM_RE(x_cali_g, params_GYM.loc[basin].values)
    _, e_sim_ym  = mYWBMnlS_RE(x_esim_l, params_YM.loc[basin].values)
    _, e_sim_am  = abcdnlS_RE(x_esim_l, params_AM.loc[basin].values)
    _, e_sim_dm  = DWBMnlS_RE(x_esim_l, params_DM.loc[basin].values)
    _, e_sim_gym = GmYWBM_RE(x_esim_g, params_GYM.loc[basin].values)
    y_mask = ~np.isnan(y_cali_l) & ~np.isnan(r_sim_ym) & ~np.isnan(r_sim_am) & ~np.isnan(r_sim_dm)
    y_cali_l  = y_cali_l[y_mask]
    r_sim_ym  = r_sim_ym[y_mask]
    r_sim_am  = r_sim_am[y_mask]
    r_sim_dm  = r_sim_dm[y_mask]
    r_sim_gym = r_sim_gym[y_mask]
    y_mask = ~np.isnan(y_esim_l) & ~np.isnan(e_sim_ym) & ~np.isnan(e_sim_am) & ~np.isnan(e_sim_dm)
    y_esim_l  = y_esim_l[y_mask]
    e_sim_ym  = e_sim_ym[y_mask]
    e_sim_am  = e_sim_am[y_mask]
    e_sim_dm  = e_sim_dm[y_mask]
    e_sim_gym = e_sim_gym[y_mask]

    Rsim = np.vstack([r_sim_ym, r_sim_am, r_sim_dm, r_sim_gym]).T
    Esim = np.vstack([e_sim_ym, e_sim_am, e_sim_dm, e_sim_gym]).T

    Rsim = np.where(np.isinf(Rsim), np.nan, Rsim)  # 把 inf 换成 nan
    row_means = np.nanmean(Rsim, axis=1)  # 每行的非nan平均
    inds = np.where(np.isnan(Rsim))
    Rsim[inds] = np.take(row_means, inds[0])  # 用对应行的均值替换nan
    Esim = np.where(np.isinf(Esim), np.nan, Esim)  # 把 inf 换成 nan
    row_means = np.nanmean(Esim, axis=1)  # 每行的非nan平均
    inds = np.where(np.isnan(Esim))
    Esim[inds] = np.take(row_means, inds[0])  # 用对应行的均值替换nan

    Robs = y_cali_l
    weights_grb = grb_weights(Rsim, Robs)
    model_names = ['YM','AM','DM','GYM']
    weight_GRB = dict(zip(model_names, weights_grb))

    w_ic_ym  = weight_GRB['YM']
    w_ic_am  = weight_GRB['AM']
    w_ic_dm  = weight_GRB['DM']
    w_ic_gym = weight_GRB['GYM']
    results['r_w_YM'][b]  = w_ic_ym
    results['r_w_AM'][b]  = w_ic_am
    results['r_w_DM'][b]  = w_ic_dm
    results['r_w_GYM'][b] = w_ic_gym

    Eobs = y_esim_l
    weights_grb = grb_weights(Esim, Eobs)
    model_names = ['YM','AM','DM','GYM']
    weight_GRB = dict(zip(model_names, weights_grb))
    w_ic_ym  = weight_GRB['YM']
    w_ic_am  = weight_GRB['AM']
    w_ic_dm  = weight_GRB['DM']
    w_ic_gym = weight_GRB['GYM']
    results['e_w_YM'][b]  = w_ic_ym
    results['e_w_AM'][b]  = w_ic_am
    results['e_w_DM'][b]  = w_ic_dm
    results['e_w_GYM'][b] = w_ic_gym

df_results = pd.DataFrame(results, index=basin_list)
df_results.index.name = 'stat_num'
df_results.to_csv("../../Results/Weighted_Average/Weighted_Average_Results_GRB.txt", sep = '\t', header=True, float_format="%.3f", index=True)

  0%|          | 0/2003 [00:00<?, ?it/s]

In [17]:
result_vars = ['r_w', 'e_w']
models = ['YM', 'AM', 'DM']
results = {f"{var}_{model}": np.full(2003, np.nan) for var in result_vars for model in models}

for b in trange(len(basin_list)):
    basin = str(basin_list[b])
    # print(f"Working: {b+1}. {basin}")
    ## 流域数据读取
    # 径流模拟数据
    x_cali_l, y_cali_l, _, _, x_esim_l, y_esim_l = get_data_lumped(basin, b)

    # 径流模拟
    r_sim_ym, _  = mYWBMnlS_RE(x_cali_l, params_YM.loc[basin].values)
    r_sim_am, _  = abcdnlS_RE(x_cali_l, params_AM.loc[basin].values)
    r_sim_dm, _  = DWBMnlS_RE(x_cali_l, params_DM.loc[basin].values)

    _, e_sim_ym  = mYWBMnlS_RE(x_esim_l, params_YM.loc[basin].values)
    _, e_sim_am  = abcdnlS_RE(x_esim_l, params_AM.loc[basin].values)
    _, e_sim_dm  = DWBMnlS_RE(x_esim_l, params_DM.loc[basin].values)

    y_mask = ~np.isnan(y_cali_l) & ~np.isnan(r_sim_ym) & ~np.isnan(r_sim_am) & ~np.isnan(r_sim_dm)
    y_cali_l  = y_cali_l[y_mask]
    r_sim_ym  = r_sim_ym[y_mask]
    r_sim_am  = r_sim_am[y_mask]
    r_sim_dm  = r_sim_dm[y_mask]

    y_mask = ~np.isnan(y_esim_l) & ~np.isnan(e_sim_ym) & ~np.isnan(e_sim_am) & ~np.isnan(e_sim_dm)
    y_esim_l  = y_esim_l[y_mask]
    e_sim_ym  = e_sim_ym[y_mask]
    e_sim_am  = e_sim_am[y_mask]
    e_sim_dm  = e_sim_dm[y_mask]


    Rsim = np.vstack([r_sim_ym, r_sim_am, r_sim_dm]).T
    Esim = np.vstack([e_sim_ym, e_sim_am, e_sim_dm]).T

    Rsim = np.where(np.isinf(Rsim), np.nan, Rsim)  # 把 inf 换成 nan
    row_means = np.nanmean(Rsim, axis=1)  # 每行的非nan平均
    inds = np.where(np.isnan(Rsim))
    Rsim[inds] = np.take(row_means, inds[0])  # 用对应行的均值替换nan
    Esim = np.where(np.isinf(Esim), np.nan, Esim)  # 把 inf 换成 nan
    row_means = np.nanmean(Esim, axis=1)  # 每行的非nan平均
    inds = np.where(np.isnan(Esim))
    Esim[inds] = np.take(row_means, inds[0])  # 用对应行的均值替换nan

    Robs = y_cali_l
    weights_grb = grb_weights(Rsim, Robs)
    model_names = ['YM','AM','DM']
    weight_GRB = dict(zip(model_names, weights_grb))

    w_ic_ym  = weight_GRB['YM']
    w_ic_am  = weight_GRB['AM']
    w_ic_dm  = weight_GRB['DM']

    results['r_w_YM'][b]  = w_ic_ym
    results['r_w_AM'][b]  = w_ic_am
    results['r_w_DM'][b]  = w_ic_dm

    Eobs = y_esim_l
    weights_grb = grb_weights(Esim, Eobs)
    model_names = ['YM','AM','DM']
    weight_GRB = dict(zip(model_names, weights_grb))
    w_ic_ym  = weight_GRB['YM']
    w_ic_am  = weight_GRB['AM']
    w_ic_dm  = weight_GRB['DM']

    results['e_w_YM'][b]  = w_ic_ym
    results['e_w_AM'][b]  = w_ic_am
    results['e_w_DM'][b]  = w_ic_dm

df_results = pd.DataFrame(results, index=basin_list)
df_results.index.name = 'stat_num'
df_results.to_csv("../../Results/Weighted_Average/Weighted_Average_Results_GRB_3M.txt", sep = '\t', header=True, float_format="%.3f", index=True)

  0%|          | 0/2003 [00:00<?, ?it/s]

# GRA方法计算多模型平均

In [19]:
def gra_weights(X, y):
    X_ext = np.column_stack([np.ones(X.shape[0]), X])  # 加一列截距项
    beta = np.linalg.lstsq(X_ext, y, rcond=None)[0]
    a_hat = beta[0]
    w_hat = beta[1:]
    y_pred = X_ext.dot(beta)
    sse = np.sum((y - y_pred) ** 2)
    return a_hat, w_hat, y_pred, sse

In [20]:
result_vars = ['r_w', 'e_w']
models = ['YM', 'AM', 'DM', 'GYM']
results = {f"{var}_{model}": np.full(2003, np.nan) for var in result_vars for model in models}

for b in trange(len(basin_list)):
    basin = str(basin_list[b])
    # print(f"Working: {b+1}. {basin}")
    ## 流域数据读取
    # 径流模拟数据
    x_cali_l, y_cali_l, _, _, x_esim_l, y_esim_l = get_data_lumped(basin, b)
    x_cali_g, y_cali_g, _, _, x_esim_g, y_esim_g = get_data_distributed(basin, b)
    # 径流模拟
    r_sim_ym, _  = mYWBMnlS_RE(x_cali_l, params_YM.loc[basin].values)
    r_sim_am, _  = abcdnlS_RE(x_cali_l, params_AM.loc[basin].values)
    r_sim_dm, _  = DWBMnlS_RE(x_cali_l, params_DM.loc[basin].values)
    r_sim_gym, _ = GmYWBM_RE(x_cali_g, params_GYM.loc[basin].values)
    _, e_sim_ym  = mYWBMnlS_RE(x_esim_l, params_YM.loc[basin].values)
    _, e_sim_am  = abcdnlS_RE(x_esim_l, params_AM.loc[basin].values)
    _, e_sim_dm  = DWBMnlS_RE(x_esim_l, params_DM.loc[basin].values)
    _, e_sim_gym = GmYWBM_RE(x_esim_g, params_GYM.loc[basin].values)
    y_mask = ~np.isnan(y_cali_l) & ~np.isnan(r_sim_ym) & ~np.isnan(r_sim_am) & ~np.isnan(r_sim_dm)
    y_cali_l  = y_cali_l[y_mask]
    r_sim_ym  = r_sim_ym[y_mask]
    r_sim_am  = r_sim_am[y_mask]
    r_sim_dm  = r_sim_dm[y_mask]
    r_sim_gym = r_sim_gym[y_mask]
    y_mask = ~np.isnan(y_esim_l) & ~np.isnan(e_sim_ym) & ~np.isnan(e_sim_am) & ~np.isnan(e_sim_dm)
    y_esim_l  = y_esim_l[y_mask]
    e_sim_ym  = e_sim_ym[y_mask]
    e_sim_am  = e_sim_am[y_mask]
    e_sim_dm  = e_sim_dm[y_mask]
    e_sim_gym = e_sim_gym[y_mask]

    Rsim = np.vstack([r_sim_ym, r_sim_am, r_sim_dm, r_sim_gym]).T
    Esim = np.vstack([e_sim_ym, e_sim_am, e_sim_dm, e_sim_gym]).T

    Rsim = np.where(np.isinf(Rsim), np.nan, Rsim)  # 把 inf 换成 nan
    row_means = np.nanmean(Rsim, axis=1)  # 每行的非nan平均
    inds = np.where(np.isnan(Rsim))
    Rsim[inds] = np.take(row_means, inds[0])  # 用对应行的均值替换nan
    Esim = np.where(np.isinf(Esim), np.nan, Esim)  # 把 inf 换成 nan
    row_means = np.nanmean(Esim, axis=1)  # 每行的非nan平均
    inds = np.where(np.isnan(Esim))
    Esim[inds] = np.take(row_means, inds[0])  # 用对应行的均值替换nan

    Robs = y_cali_l
    a_gra, weights_gra, R_gra, sse = gra_weights(Rsim, Robs)
    R_GRA = a_gra + Rsim.dot(weights_gra)
    model_names = ['YM','AM','DM','GYM']
    weight_GRA = dict(zip(model_names, weights_gra.tolist()))

    w_ic_ym  = weight_GRA['YM']
    w_ic_am  = weight_GRA['AM']
    w_ic_dm  = weight_GRA['DM']
    w_ic_gym = weight_GRA['GYM']
    results['r_w_YM'][b]  = w_ic_ym
    results['r_w_AM'][b]  = w_ic_am
    results['r_w_DM'][b]  = w_ic_dm
    results['r_w_GYM'][b] = w_ic_gym

    Eobs = y_esim_l
    a_gra, weights_gra, E_gra, sse = gra_weights(Esim, Eobs)
    E_GRA = a_gra + Esim.dot(weights_gra)
    model_names = ['YM','AM','DM','GYM']
    weight_GRA = dict(zip(model_names, weights_gra.tolist()))
    w_ic_ym  = weight_GRA['YM']
    w_ic_am  = weight_GRA['AM']
    w_ic_dm  = weight_GRA['DM']
    w_ic_gym = weight_GRA['GYM']
    results['e_w_YM'][b]  = w_ic_ym
    results['e_w_AM'][b]  = w_ic_am
    results['e_w_DM'][b]  = w_ic_dm
    results['e_w_GYM'][b] = w_ic_gym

df_results = pd.DataFrame(results, index=basin_list)
df_results.index.name = 'stat_num'
df_results.to_csv("../../Results/Weighted_Average/Weighted_Average_Results_GRA.txt", sep = '\t', header=True, float_format="%.3f", index=True)

  0%|          | 0/2003 [00:00<?, ?it/s]

In [21]:
for b in trange(len(basin_list)):
    basin = str(basin_list[b])
    # print(f"Working: {b+1}. {basin}")
    ## 流域数据读取
    # 径流模拟数据
    x_cali_l, y_cali_l, _, _, x_esim_l, y_esim_l = get_data_lumped(basin, b)
    # 径流模拟
    r_sim_ym, _  = mYWBMnlS_RE(x_cali_l, params_YM.loc[basin].values)
    r_sim_am, _  = abcdnlS_RE(x_cali_l, params_AM.loc[basin].values)
    r_sim_dm, _  = DWBMnlS_RE(x_cali_l, params_DM.loc[basin].values)

    _, e_sim_ym  = mYWBMnlS_RE(x_esim_l, params_YM.loc[basin].values)
    _, e_sim_am  = abcdnlS_RE(x_esim_l, params_AM.loc[basin].values)
    _, e_sim_dm  = DWBMnlS_RE(x_esim_l, params_DM.loc[basin].values)

    y_mask = ~np.isnan(y_cali_l) & ~np.isnan(r_sim_ym) & ~np.isnan(r_sim_am) & ~np.isnan(r_sim_dm)
    y_cali_l  = y_cali_l[y_mask]
    r_sim_ym  = r_sim_ym[y_mask]
    r_sim_am  = r_sim_am[y_mask]
    r_sim_dm  = r_sim_dm[y_mask]

    y_mask = ~np.isnan(y_esim_l) & ~np.isnan(e_sim_ym) & ~np.isnan(e_sim_am) & ~np.isnan(e_sim_dm)
    y_esim_l  = y_esim_l[y_mask]
    e_sim_ym  = e_sim_ym[y_mask]
    e_sim_am  = e_sim_am[y_mask]
    e_sim_dm  = e_sim_dm[y_mask]

    Rsim = np.vstack([r_sim_ym, r_sim_am, r_sim_dm]).T
    Esim = np.vstack([e_sim_ym, e_sim_am, e_sim_dm]).T

    Rsim = np.where(np.isinf(Rsim), np.nan, Rsim)  # 把 inf 换成 nan
    row_means = np.nanmean(Rsim, axis=1)  # 每行的非nan平均
    inds = np.where(np.isnan(Rsim))
    Rsim[inds] = np.take(row_means, inds[0])  # 用对应行的均值替换nan
    Esim = np.where(np.isinf(Esim), np.nan, Esim)  # 把 inf 换成 nan
    row_means = np.nanmean(Esim, axis=1)  # 每行的非nan平均
    inds = np.where(np.isnan(Esim))
    Esim[inds] = np.take(row_means, inds[0])  # 用对应行的均值替换nan

    Robs = y_cali_l
    a_gra, weights_gra, R_gra, sse = gra_weights(Rsim, Robs)
    R_GRA = a_gra + Rsim.dot(weights_gra)
    model_names = ['YM','AM','DM']
    weight_GRA = dict(zip(model_names, weights_gra.tolist()))

    w_ic_ym  = weight_GRA['YM']
    w_ic_am  = weight_GRA['AM']
    w_ic_dm  = weight_GRA['DM']

    results['r_w_YM'][b]  = w_ic_ym
    results['r_w_AM'][b]  = w_ic_am
    results['r_w_DM'][b]  = w_ic_dm

    Eobs = y_esim_l
    a_gra, weights_gra, E_gra, sse = gra_weights(Esim, Eobs)
    E_GRA = a_gra + Esim.dot(weights_gra)
    model_names = ['YM','AM','DM']
    weight_GRA = dict(zip(model_names, weights_gra.tolist()))
    w_ic_ym  = weight_GRA['YM']
    w_ic_am  = weight_GRA['AM']
    w_ic_dm  = weight_GRA['DM']

    results['e_w_YM'][b]  = w_ic_ym
    results['e_w_AM'][b]  = w_ic_am
    results['e_w_DM'][b]  = w_ic_dm

df_results = pd.DataFrame(results, index=basin_list)
df_results.index.name = 'stat_num'
df_results.to_csv("../../Results/Weighted_Average/Weighted_Average_Results_GRA_3M.txt", sep = '\t', header=True, float_format="%.3f", index=True)

  0%|          | 0/2003 [00:00<?, ?it/s]